In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pickle

import torch

import hydra
from hydra import initialize, compose

from home_robot.mapping.voxel.feature_voxel import ConceptFusion

In [ ]:
from home_robot.datasets.scannet import ScanNetDataset, ReferIt3dDataConfig, ScanReferDataConfig
data = ScanNetDataset(
    root_dir = '/private/home/ssax/home-robot/src/home_robot/home_robot/datasets/scannet/data',
    frame_skip = 180,
    n_classes = 50,
    referit3d_config = ReferIt3dDataConfig(),
    scanrefer_config = ScanReferDataConfig(),
)

# Load specific scene
# scene0192_00 -- small scene
# 'scene0000_00' -- large scene
idx = data.scene_list.index("scene0192_00") #'scene0000_00'
# idx = 0
print(f"Loaded images of (h: {data.height}, w: {data.width}) - resized from ({data.DEFAULT_HEIGHT},{data.DEFAULT_WIDTH})")
scene_obs = data.__getitem__(idx, show_progress=True)
torch.autograd.set_grad_enabled(False)


In [ ]:
# load parameters
hydra.core.global_hydra.GlobalHydra.instance().clear()

# hydra.initialize(config_path='configs', config_name='cf_modular')

with initialize(version_base='1.3', config_path="configs/model"):
    cfg = compose(config_name="conceptfusion", overrides=[])

# Initialize ConceptFusion only once
concept_fusion = hydra.utils.instantiate(cfg)

In [ ]:
concept_fusion.clear()


# SETUP EVAL
class_id_to_class_names = dict(
    zip(
        data.METAINFO["CLASS_IDS"],  # IDs [1, 3, 4, 5, ..., 65]
        data.METAINFO["CLASS_NAMES"],  # [wall, floor, cabinet, ...]
    )
)

concept_fusion.set_vocabulary(class_id_to_class_names)

keys = [
    "images",
    "depths",
    "poses",
    "intrinsics",
    "boxes_aligned",
    "box_classes",
]

gt_bounds, gt_classes, pred_bounds, pred_classes, pred_scores = [], [], [], [], []
# Move to device
for k in keys:
    scene_obs[k] = scene_obs[k].to(concept_fusion.device)

# Eval each scene and move to CPU
queries = {
    int(clas): class_id_to_class_names[int(clas)]
    for clas in scene_obs["box_classes"].unique()
}

# concept_fusion.build_scene(scene_obs)
# concept_fusion.show_point_cloud_pytorch3d()

instances_dict = concept_fusion.build_scene_and_get_instances_for_queries(scene_obs, queries.values())
concept_fusion.show_point_cloud_pytorch3d(instances_dict)

In [ ]:
pc_xyz, pc_feat, _, pc_rgb = concept_fusion.voxel_map.get_pointcloud()
inds, similarity = concept_fusion.query_similarity("chair", pc_feat)


In [ ]:
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs
from home_robot.utils.bboxes_3d import BBoxes3D

from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from home_robot.utils.data_tools.dict import update
import matplotlib
traces = {}
cmap = matplotlib.colormaps["jet"]
similarity_colormap = cmap(similarity.detach().cpu().numpy())[:, :3]

map_colors = 0.5 * pc_rgb.cpu() / 255. + 0.5 * torch.tensor(similarity_colormap)
map_colors = torch.tensor(similarity_colormap)
traces["Points"] = Pointclouds(points=[pc_xyz.cpu()], features=[map_colors])

_default_plot_args = dict(
            xaxis={"backgroundcolor": "rgb(200, 200, 230)"},
            yaxis={"backgroundcolor": "rgb(230, 200, 200)"},
            zaxis={"backgroundcolor": "rgb(200, 230, 200)"},
            axis_args=AxisArgs(showgrid=True),
            pointcloud_marker_size=3,
            pointcloud_max_points=500_000,
            boxes_plot_together=False,
            boxes_wireframe_width=3,
)
fig = plot_scene_with_bboxes(
            plots={f"Conceptfusion Pointcloud": traces},
            **update(_default_plot_args, _default_plot_args),
)
fig.update_layout(
            height=800,
            width=1600,
)
fig

In [ ]:
instances_dict = concept_fusion.get_instances_for_queries(queries.values())
concept_fusion.show_point_cloud_pytorch3d(instances_dict)

In [ ]:
# Block to test hyperparams.

# remove "otherfurniture" from queries to test performance without it
# All furniture classes have high similarity to "otherfurniture" so it is not a good query
queries.pop(39)

concept_fusion.dbscan_params.epsilon = 0.2
concept_fusion.dbscan_params.min_samples = 50
concept_fusion.similarity_params.similarity_thresh = 0.2
instances_dict = concept_fusion.text_queries(queries=queries.values())
concept_fusion.show_point_cloud_pytorch3d(instances_dict)

In [ ]:
# Visualize how the ground truth boxes look like
import open3d as o3d
import numpy as np

pcd = o3d.io.read_point_cloud("/srv/flash1/kyadav32/datasets/ovmm/scannet/scannet_scene0011_00_gt.ply")

from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs
from home_robot.utils.bboxes_3d import BBoxes3D

from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from home_robot.utils.data_tools.dict import update

from utils import COLOR_LIST

traces = {}

pc_xyz, pc_rgb = pcd.points, pcd.colors
pc_xyz = torch.tensor(np.asarray(pc_xyz), device="cuda")
pc_rgb = torch.tensor(np.asarray(pc_rgb), device="cuda")

traces["Points"] = Pointclouds(points=[pc_xyz], features=[pc_rgb*255])


box_classes = dataset_0[0]['box_classes']
box_bounds = dataset_0[0]['boxes_aligned']

bounds, names, colors = {}, {}, {}
for class_id, bound in zip(box_classes, box_bounds):
    
    class_name = class_id_to_class_names[class_id.item()]

    if class_name not in bounds:
        bounds[class_name] = []
        names[class_name] = []
        colors[class_name] = []
    bounds[class_name].append(bound)
    names[class_name].append(torch.tensor(class_id, device='cuda'))
    colors[class_name].append(torch.tensor(COLOR_LIST[class_id % len(COLOR_LIST)], device='cuda'))
for class_name in box_classes.keys():
    detected_boxes = BBoxes3D(
        bounds=[torch.stack(bounds, dim=0)],
        features=[torch.stack(colors, dim=0)],
        names=[torch.stack(names, dim=0).unsqueeze(-1)],
    )
    traces[class_name + "_bbox"] = detected_boxes


_default_plot_args = dict(
    xaxis={"backgroundcolor": "rgb(200, 200, 230)"},
    yaxis={"backgroundcolor": "rgb(230, 200, 200)"},
    zaxis={"backgroundcolor": "rgb(200, 230, 200)"},
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=500_000,
    boxes_plot_together=False,
    boxes_wireframe_width=3,
)
fig = plot_scene_with_bboxes(
    plots={f"Conceptfusion Pointcloud": traces},
    **update(_default_plot_args, {}),
)
fig.update_layout(
    height=800,
    width=1600,
)
